In [1]:
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, summarize_and_visualize

ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2023-12-10 08:00"
END_DATE = "2024-12-12 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}
transfers_df = fetch_and_save_erc20_transfers(
        address=ADDRESS,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        offset=1000,
    )
# final_record_df  = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# summary_df = summarize_and_visualize(final_record_df, BASE_TOKENS)

2024-12-23 17:41:22,029 - INFO - Start block: 18752097, End block: 21382703
2024-12-23 17:41:22,029 - INFO - Fetching ERC20 transfers from block 18752097 to 21382703...
2024-12-23 17:41:23,731 - INFO - Page 1: Retrieved 1000 transactions.
2024-12-23 17:41:25,415 - INFO - Page 2: Retrieved 1000 transactions.
2024-12-23 17:41:27,084 - INFO - Page 3: Retrieved 1000 transactions.
2024-12-23 17:41:28,817 - INFO - Page 4: Retrieved 1000 transactions.
2024-12-23 17:41:30,218 - INFO - Page 5: Retrieved 263 transactions.
2024-12-23 17:41:31,153 - WARNING - API request failed on page 6: Request failed: No transactions found
2024-12-23 17:41:31,153 - INFO - Final dataset contains 4263 unique transactions.
2024-12-23 17:41:31,224 - INFO - Data successfully saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result\erc20_transfers.csv
2024-12-23 17:41:31,225 - INFO - Transfers saved to C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result
